In [1]:
import h5py, cv2, os, sys
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

sys.path.append("../")


In [2]:
pkl_path = "../data/ecommerce_cleaned.pkl"
df = pd.read_pickle(filepath_or_buffer=pkl_path)


In [3]:
df.head(n=1)


,product_name,doc_desc,lem_desc,first_category,product_specifications,image,description,category_tree
0,Elegance Polyester Multicolor Abstract Eyelet ...,"(key, elegance, polyester, multicolor, abstrac...","[key, elegance, polyester, multicolor, abstrac...",home furnishing,"{""product_specification""=>[{""key""=>""Brand"", ""v...",55b85ea15a1536d46b7190ad6fff8ce7.jpg,Key Features of Elegance Polyester Multicolor ...,"[Home Furnishing, Curtains & Accessories, Curt..."


In [4]:
df.columns

Index(['product_name', 'doc_desc', 'lem_desc', 'first_category',
       'product_specifications', 'image', 'description', 'category_tree'],
      dtype='object')

In [5]:
# We want to keep : image and first category 
keepcols = ["image", "first_category"]
df = df[keepcols]


In [6]:
df.rename(columns={"first_category": "category", "image": "image_name"}, inplace=True)


In [7]:
df.head()


,image_name,category
0,55b85ea15a1536d46b7190ad6fff8ce7.jpg,home furnishing
1,7b72c92c2f6c40268628ec5f14c6d590.jpg,baby care
2,d4684dcdc759dd9cdf41504698d737d8.jpg,home furnishing
3,6325b6870c54cd47be6ebfbffa620ec7.jpg,home furnishing
4,893aa5ed55f7cff2eccea7758d7a86bd.jpg,watches


In [8]:
image_dir = "../imgs/vgg_16/"
df_train, df_test = train_test_split(
    df,
    test_size=0.3,
    stratify=df["category"]  # ensure category harmony
    )

df_test.reset_index(inplace=True)  # resetting both indexes before iterating over them
df_train.reset_index(inplace=True)


In [14]:
height = 224
width = height
channel = 3  # RGB

with h5py.File("../data/product_images.h5", "w") as h5_file:
    image_shape_train = (len(df_train), height, width, channel)
    image_shape_test = (len(df_test), height, width, channel)

    images_train = h5_file.create_dataset("images_train", image_shape_train, dtype="uint8")
    categories_train = h5_file.create_dataset("category_train", (len(df_train), ), dtype=h5py.special_dtype(vlen=str))

    images_test = h5_file.create_dataset("images_test", image_shape_test, dtype="uint8")
    categories_test = h5_file.create_dataset("category_test", (len(df_test), ), dtype=h5py.special_dtype(vlen=str))

    for index, row in df_train.iterrows():
        image = cv2.imread(image_dir + row["image_name"])
        images_train[index] = image
        categories_train[index] = row["category"]

    for index, row in df_test.iterrows():
        image = cv2.imread(image_dir + row["image_name"])
        images_test[index] = image
        categories_test[index] = row["category"]


In [15]:
with h5py.File("../data/product_images.h5") as h5:
    print(h5.keys())


<KeysViewHDF5 ['category_test', 'category_train', 'images_test', 'images_train']>
